In [2]:
import pandas
import os
import sys
import numpy as np
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.initializers import Constant


Using TensorFlow backend.


In [1]:


MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2


In [3]:
BASE_DIR = '../'
GLOVE_DIR = os.path.join(BASE_DIR, 'glove.6B')


In [4]:
print('Indexing word vectors.')

embeddings_index = {}
with open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt')) as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))


Indexing word vectors.
Found 400000 word vectors.


In [5]:
# second, prepare text samples and their labels
print('Processing text dataset')


df = pandas.read_csv('processedQuestions.csv')
d = df.to_dict()

docs = []
labels = []
for key2 in d['Subject']:
	if pandas.isna(d['Subject'][key2]) or pandas.isna(d['Content'][key2]) or pandas.isna(d['Solution'][key2]) :
		pass
	else:
		if (d['Subject'][key2].replace('\"','').strip(" ") == "Physics"):
			labels.append(0)
		elif (d['Subject'][key2].replace('\"','').strip(" ") == "Chemistry"):
			labels.append(1)
		else :
			labels.append(2)

		# Need some good string parsing here
		strin = d['Content'][key2] + d['Solution'][key2]
		docs.append( strin )


Processing text dataset


In [6]:
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(docs)
sequences = tokenizer.texts_to_sequences(docs)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)


Found 26537 unique tokens.
('Shape of data tensor:', (16975, 1000))
('Shape of label tensor:', (16975, 3))


In [7]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]


In [8]:
print('Preparing embedding matrix.')

# prepare embedding matrix
num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i > MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
        embedding_matrix[i] = np.random.random_sample((100))

Preparing embedding matrix.


In [9]:
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)


In [10]:
print('Training model.')

# train a 1D convnet with global maxpooling
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = GlobalMaxPooling1D()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(3, activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
                optimizer='rmsprop',
                metrics=['acc'])

print(model.summary())


Training model.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1000, 100)         2000100   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 996, 128)          64128     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 199, 128)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 195, 128)          82048     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 39, 128)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 35, 128)           82048

In [11]:
model.fit(x_train, y_train,
            batch_size=128,
            epochs=15,
            validation_data=(x_val, y_val))


Train on 13580 samples, validate on 3395 samples
Epoch 1/15
13580/13580 [==============================] - 251s 18ms/step - loss: 0.6792 - acc: 0.7225 - val_loss: 0.3639 - val_acc: 0.8792
Epoch 2/15
13580/13580 [==============================] - 253s 19ms/step - loss: 0.2929 - acc: 0.9005 - val_loss: 0.3214 - val_acc: 0.8960
Epoch 3/15
13580/13580 [==============================] - 253s 19ms/step - loss: 0.1944 - acc: 0.9361 - val_loss: 0.5941 - val_acc: 0.8059
Epoch 4/15
13580/13580 [==============================] - 250s 18ms/step - loss: 0.1371 - acc: 0.9532 - val_loss: 0.4548 - val_acc: 0.8242
Epoch 5/15
13580/13580 [==============================] - 146s 11ms/step - loss: 0.1017 - acc: 0.9655 - val_loss: 0.2243 - val_acc: 0.9402
Epoch 6/15
13580/13580 [==============================] - 136s 10ms/step - loss: 0.0770 - acc: 0.9733 - val_loss: 0.2460 - val_acc: 0.9487
Epoch 7/15
13580/13580 [==============================] - 161s 12ms/step - loss: 0.0500 - acc: 0.9842 - val_loss: 0.4

In [12]:
model.save('topic102.h5')
dump(tokenizer, open('Topictokenizer2.pkl', 'wb'))


In [18]:
str = raw_input('Enter Your query to classify\n')

docs = [str]
sequences = tokenizer.texts_to_sequences(docs)
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
np.set_printoptions(suppress=True)
output = model.predict(data,verbose=0)

print(output.flatten())



Enter Your query to classify
Which of the following reagents may be employed to distinguish the constituents of a mixture of a water-insoluble phenol and benzoic acid?
[0. 1. 0.]


In [ ]:
[0.86389637 0.00558657 0.13051705]